In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

In [ ]:
kind = ["alex_hires","alex_lowres"]

dfs = [ pd.read_csv("now/hires/alex_test.prototxt_scores.txt"),
        pd.read_csv("now/lores/alex_test.prototxtxt_lowres_scores.txt")]

In [ ]:
def amax(row):
    return int(np.argmax(row[1:6]))

for df in dfs:
    
    softmaxes = []
    for ix, row in df.iterrows():
        ex = np.exp(row[1:6])
        softmaxes.append(np.array(ex / np.sum(ex)))
    
    df['infer'] = df.apply(amax,axis=1)
    df['softmax'] = pd.Series(softmaxes)

In [ ]:
def top_prob(row):
    return np.max(row['softmax'])

for df in dfs:
    df['top_prob'] = df.apply(top_prob, axis=1)

In [ ]:
particles = ['Eminus','Gamma','Muminus','Piminus','Proton']
pars = ['','Eminus','Gamma','Muminus','Piminus','Proton']
particles_df = {}


for ix,p in enumerate(particles):
    particles_df[p] = dfs[1].query('gt == {}'.format(ix))


In [ ]:
matplotlib.rcParams['font.size'] = 16
freqs = []

for par in particles:
    fig, ax = plt.subplots(figsize=(10,6))
    dec = []
    n = float(particles_df[par].index.size)
    for ix,p in enumerate(particles):
        dec.append(particles_df[par].query('infer == %d' % ix).index.size / n)

    ax.hist(np.arange(0,5.0,1.0),
            bins=np.arange(0,6.0,1.0),
            weights=np.array(dec)*100.0,label='True {}'.format(par,int(n)))
    
    freqs.append(dec)
    
    ax.set_ylim(0,100)
    ax.yaxis.grid()
    ax.set_ylabel("Classified %",fontweight='bold')

    ax.set_xticks(np.arange(0.5,5.5,1.0))
    ax.set_xticklabels(particles)
    ax.set_title("LowRes Image")
 
    ax.legend(loc='best')
    plt.savefig('lowres_single_plane_%d_class_%s_%s.pdf' %(len(particles),kind[0],par), format='pdf', dpi=1000)
    plt.show()

In [ ]:
particles = ['Eminus','Gamma','Muminus','Piminus','Proton']
pars = ['','Eminus','Gamma','Muminus','Piminus','Proton']
particles_df = {}


for ix,p in enumerate(particles):
    particles_df[p] = dfs[0].query('gt == {}'.format(ix))

matplotlib.rcParams['font.size'] = 16
freqs = []

for par in particles:
    fig, ax = plt.subplots(figsize=(10,6))
    dec = []
    n = float(particles_df[par].index.size)
    for ix,p in enumerate(particles):
        dec.append(particles_df[par].query('infer == %d' % ix).index.size / n)

    ax.hist(np.arange(0,5.0,1.0),
            bins=np.arange(0,6.0,1.0),
            weights=np.array(dec)*100.0,label='True {}'.format(par,int(n)))
    
    freqs.append(dec)
    
    ax.set_ylim(0,100)
    ax.yaxis.grid()
    ax.set_ylabel("Classified %",fontweight='bold')

    ax.set_xticks(np.arange(0.5,5.5,1.0))
    ax.set_xticklabels(particles)
    ax.set_title("HiRes Image")
 
    ax.legend(loc='best')
    plt.savefig('hiresres_single_plane_%d_class_%s_%s.pdf' %(len(particles),kind[0],par), format='pdf', dpi=1000)
    plt.show()

In [ ]:
dfs[1]

In [ ]:
a = np.array(freqs)

fig,ax = plt.subplots(figsize=(12,8))
im = ax.matshow(np.array(a),origin='bottom')
ax.xaxis.set_ticks_position('bottom')
ax.set_xticklabels([''] + particles,rotation=45,fontweight='bold')
ax.set_yticklabels([''] + particles,fontweight='bold')
ax.set_title("Confusion Matrix",fontweight='bold')
plt.colorbar(im)
plt.tight_layout()
ax.text(4.5,2,"Percent Fraction",rotation=270,fontweight='bold')
plt.savefig('5_class_confusion_matrix_{}.pdf'.format(kind), format='pdf', dpi=1000)
plt.show()

In [ ]:
ar = lambda x, y : np.arange(0,x,y)
matplotlib.rcParams['font.size']=14
matplotlib.rcParams['font.family']='serif'

In [ ]:
inc = 0.01

npars = len(pars) - 1

for pred in xrange(npars):
    a = {}
    fig,ax = plt.subplots(figsize=(12,6))
    model_df = df

    for i in xrange(npars):
        res = []
        for j in ar(1,inc):
            bot = model_df[model_df['infer'] == pred].query('top_prob > {}'.format(j))
            top = bot.query('gt == {}'.format(float(i)))
            bot = float(bot.index.size)
            top = float(top.index.size)
            r = top/bot if bot > 0.0 else 0.0
            res.append(r * 100.0)
            a[i] = res

    ax.hist([ar(1,inc) for _ in range(npars)],bins=ar(1+inc,inc),
            weights=[a[r] for r in a],
            stacked=True,histtype='stepfilled',alpha=0.65,label=[particles[r] for r in a])
    ax.set_ylim(0,100)
    ax.legend(loc='best',fontsize=12)

    ax.set_title("Classification: {} - Model: {}".format(particles[pred],kind),fontweight='bold')
    ax.set_ylabel("Percent Truth",fontweight='bold')
    ax.set_xlabel("Prob. Threshold",fontweight='bold')
    plt.savefig('{}_alex_classification_truth_{}_{}.pdf'.format(kind,particles[pred],npars), format='pdf', dpi=1000)
    plt.show()

In [ ]:
inc = 0.01

npars = len(pars) - 1

for pred in xrange(npars):
    a = {}
    fig,ax = plt.subplots(figsize=(12,6))
    model_df = df

    for i in xrange(npars):
        res = []
        for j in ar(1,inc):
            bot = model_df[model_df['infer'] == pred].query('top_prob > {}'.format(j))
            top = bot.query('gt == {}'.format(i))
            bot = float(bot.index.size)
            top = float(top.index.size)
            r = top/bot if bot > 0.0 else 0.0
            res.append(r * 100.0)
            a[i] = res

    ax.hist([ar(1,inc) for _ in range(npars)],bins=ar(1+inc,inc),
            weights=[a[r] for r in a],
            stacked=True,histtype='stepfilled',alpha=0.65,label=[particles[r] for r in a])
    ax.set_ylim(0,100)
    ax.legend(loc='best',fontsize=12)

    ax.set_title("Truth: {} - Model: {}".format(particles[pred],kind),fontweight='bold')
    ax.set_ylabel("Percent Classified",fontweight='bold')
    ax.set_xlabel("Prob. Threshold",fontweight='bold')
    plt.savefig('{}_alex_truth_classification{}_{}.pdf'.format(kind,particles[pred],npars), format='pdf', dpi=1000)
    plt.show()

In [ ]:
inc = 0.01

npars = len(pars) - 1

for pred in xrange(npars):
    a = {}
    fig,ax = plt.subplots(figsize=(12,6))
    model_df = df
    b = None

    for i in xrange(npars):
        res = []
        b = []
        for j in ar(1,inc):
            bot = model_df[model_df['gt'] == pred]
            top = bot.query('infer == {} & top_prob > {}'.format(i,j))
            bot = float(bot.index.size)
            top = float(top.index.size)
            r = top/bot if bot > 0.0 else 0.0
            res.append(r * 100.0)
            a[i] = res

        #ax.plot(ar(1,inc),np.array(a[i]),lw=2,label=particles[i])
       
    ax.hist([ar(1,inc) for _ in range(npars)],bins=ar(1+inc,inc),
            weights=[a[r] for r in a],
            stacked=True,histtype='stepfilled',alpha=0.65,label=[particles[r] for r in a])
    ax.set_ylim(0,100)
    ax.legend(loc='best',fontsize=12)

    ax.set_title("Efficiency. Truth: {} - Model: {}".format(particles[pred],kind),fontweight='bold')
    ax.set_ylabel("Percent Classified",fontweight='bold')
    ax.set_xlabel("Prob. Threshold",fontweight='bold')
    plt.savefig('efficiency_{}_alex_truth_classification{}_{}.pdf'.format(kind,particles[pred],npars), 
                format='pdf', 
                dpi=1000)
    plt.show()


In [ ]:

for ix,par in enumerate(particles):
    b = []

    for j in ar(1,inc):
        b.append(float(df.query('gt == {} and top_prob > {}'.format(ix,j)).index.size))
    
    fig,ax = plt.subplots(figsize=(12,6))
    ax.plot(ar(1,0.01),b)
    ax.set_title("Set of {} Images".format(par))
    ax.set_xlabel("Prob. Threshold",fontweight='bold')
    ax.set_ylabel("N Images Remaining",fontweight='bold')
    plt.show()